In [0]:
import pyspark.sql.functions as F

In [0]:
# Carregar a tabela do Databricks
df_databricks = spark.read.table("adventure_works_ouro.OneBigTable")


In [0]:
df_databricks = spark.read.table("adventure_works_ouro.OneBigTable")


# Configurações da conexão
jdbc_hostname = "sqldb-study-lakehouse-server.database.windows.net"
jdbc_port = 1433
database_name = "AdventureWorks2022"
jdbc_url = f"jdbc:sqlserver://{jdbc_hostname}:{jdbc_port};database={database_name}"
username = 'APRENDER_DADOS_READ' 
password = 'SW4N0FW!D0Xd4W&Rt7'  

# Configurações de autenticação
connection_properties = {
    "user": username,
    "password": password,
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

# Query para ler a tabela no banco de dados SQL Server
query = "SELECT * FROM sales.OneBigTable"

df_sql = (spark.read
    .format("jdbc")
    .option("url", jdbc_url)
    .option("query", query)
    .option("user", username)
    .option("password", password)
    .load())

    


In [0]:
# Encontrar registros no Databricks que não estão no SQL Server
missing_in_sql = df_databricks.join(
    df_sql,
    on=["SalesOrderID", "SalesOrderDetailID"],
    how="left_anti"
)

if missing_in_sql.count() > 0:
    print("Registros presentes no Databricks, mas ausentes no SQL Server:")
    missing_in_sql.show(truncate=False)
else:
    print("Nenhum registro exclusivo no Databricks.")


In [0]:
# Encontrar registros no SQL Server que não estão no Databricks
missing_in_databricks = df_sql.join(
    df_databricks,
    on=["SalesOrderID", "SalesOrderDetailID"],
    how="left_anti"
)

if missing_in_databricks.count() > 0:
    print("Registros presentes no SQL Server, mas ausentes no Databricks:")
    missing_in_databricks.show(truncate=False)
else:
    print("Nenhum registro exclusivo no SQL Server.")


In [0]:
# Fazer o join completo para comparar os valores
comparison_df = df_databricks.alias("databricks").join(
    df_sql.alias("sql"),
    on=["SalesOrderID", "SalesOrderDetailID"],
    how="inner"
)

# Encontrar diferenças nas colunas
differences = comparison_df.select(
    *[
        F.when(
            F.col(f"databricks.{col}") != F.col(f"sql.{col}"), 
            F.struct(F.col(f"databricks.{col}").alias("Databricks"), F.col(f"sql.{col}").alias("SQL Server"))
        ).alias(col)
        for col in df_databricks.columns if col not in ["SalesOrderID", "SalesOrderDetailID"]
    ]
)

# Filtrar linhas onde há diferenças
differences_df = differences.filter(
    F.greatest(*[F.col(col).isNotNull() for col in differences.columns])
)

if differences_df.count() > 0:
    print("Diferenças encontradas entre os valores das colunas:")
    differences_df.show(truncate=False)
else:
    print("Não foram encontradas diferenças nos valores das colunas.")


In [0]:
print("Colunas no DataFrame do Databricks:")
print(df_databricks.columns)

print("\nColunas no DataFrame do SQL Server:")
print(df_sql.columns)


In [0]:
# Resetar o índice para transformar as colunas do índice em colunas normais
df_sql = df_sql.reset_index(drop=True)
print("Colunas após reset do índice no DataFrame do SQL Server:")
print(df_sql.columns)


In [0]:
# Encontrar registros no Databricks que não estão no SQL Server
missing_in_sql = df_databricks.join(
    df_sql,
    on=["SalesOrderID", "SalesOrderDetailID"],
    how="left_anti"
)

if missing_in_sql.count() > 0:
    print("Registros presentes no Databricks, mas ausentes no SQL Server:")
    missing_in_sql.show(truncate=False)
else:
    print("Nenhum registro exclusivo no Databricks.")

# Encontrar registros no SQL Server que não estão no Databricks
missing_in_databricks = df_sql.join(
    df_databricks,
    on=["SalesOrderID", "SalesOrderDetailID"],
    how="left_anti"
)

if missing_in_databricks.count() > 0:
    print("Registros presentes no SQL Server, mas ausentes no Databricks:")
    missing_in_databricks.show(truncate=False)
else:
    print("Nenhum registro exclusivo no SQL Server.")


In [0]:
# Verificar se os DataFrames são iguais
comparison_result = df_databricks.equals(df_sql)

if comparison_result:
    print("Os dados do Databricks e do SQL Server são iguais.")
else:
    print("Discrepâncias encontradas entre os dados do Databricks e do SQL Server.")


In [0]:
# Comparar os DataFrames e mostrar as diferenças
differences = df_databricks.compare(df_sql)
if not differences.empty:
    print("Diferenças detalhadas:")
    print(differences)
else:
    print("Nenhuma diferença significativa encontrada.")


In [0]:
# Databricks
products_sold_databricks = (df_databricks
    .groupBy("ProductName", "ProductCategoryName", "ProductSubcategoryName")
    .agg(F.sum("OrderQty").alias("TotalQuantitySold"))
    .orderBy(F.col("TotalQuantitySold").desc())
)

# SQL Server
products_sold_sql = (df_sql
    .groupBy("ProductName", "ProductCategoryName", "ProductSubcategoryName")
    .agg(F.sum("OrderQty").alias("TotalQuantitySold"))
    .orderBy(F.col("TotalQuantitySold").desc())
)

# Mostrar os resultados
print("Top Produtos Mais Vendidos - Databricks:")
products_sold_databricks.display()

print("Top Produtos Mais Vendidos - SQL Server:")
products_sold_sql.display(

)


In [0]:
# Databricks
revenue_databricks = (df_databricks
    .groupBy("ProductName", "ProductCategoryName", "ProductSubcategoryName")
    .agg(F.sum("LineTotal").alias("TotalRevenue"))
    .orderBy(F.col("TotalRevenue").desc())
)

# SQL Server
revenue_sql = (df_sql
    .groupBy("ProductName", "ProductCategoryName", "ProductSubcategoryName")
    .agg(F.sum("LineTotal").alias("TotalRevenue"))
    .orderBy(F.col("TotalRevenue").desc())
)

# Mostrar os resultados
print("Top Produtos por Receita Total - Databricks:")
revenue_databricks.show(truncate=False)

print("Top Produtos por Receita Total - SQL Server:")
revenue_sql.show(truncate=False)


In [0]:
# Databricks
revenue_by_category_databricks = (df_databricks
    .groupBy("ProductCategoryName")
    .agg(
        F.sum("LineTotal").alias("TotalRevenue"),
        F.countDistinct("ProductName").alias("UniqueProducts")
    )
    .orderBy(F.col("TotalRevenue").desc())
)

# SQL Server
revenue_by_category_sql = (df_sql
    .groupBy("ProductCategoryName")
    .agg(
        F.sum("LineTotal").alias("TotalRevenue"),
        F.countDistinct("ProductName").alias("UniqueProducts")
    )
    .orderBy(F.col("TotalRevenue").desc())
)

# Mostrar os resultados
print("Receita por Categoria - Databricks:")
revenue_by_category_databricks.show(truncate=False)

print("Receita por Categoria - SQL Server:")
revenue_by_category_sql.show(truncate=False)


In [0]:
from pyspark.sql import functions as F

def compare_aggregations(databricks_df, sql_df, group_by_cols, agg_col, agg_func, agg_alias):
    """
    Função para comparar agregações entre Databricks e SQL Server.
    Mostra os resultados lado a lado com a diferença entre eles.
    
    Parâmetros:
    - databricks_df: DataFrame no Databricks.
    - sql_df: DataFrame do SQL Server.
    - group_by_cols: Lista de colunas de agrupamento.
    - agg_col: Coluna a ser agregada.
    - agg_func: Função de agregação (ex: F.sum).
    - agg_alias: Nome do resultado da agregação.
    """
    # Agregação no Databricks
    result_databricks = (databricks_df
        .groupBy(group_by_cols)
        .agg(agg_func(agg_col).alias(agg_alias))
    )
    
    # Agregação no SQL Server
    result_sql = (sql_df
        .groupBy(group_by_cols)
        .agg(agg_func(agg_col).alias(agg_alias))
    )
    
    # Join entre os resultados
    comparison_df = result_databricks.alias("databricks").join(
        result_sql.alias("sql"),
        on=group_by_cols,
        how="inner"
    ).select(
        *group_by_cols,
        F.col(f"databricks.{agg_alias}").alias(f"Databricks_{agg_alias}"),
        F.col(f"sql.{agg_alias}").alias(f"SQL_{agg_alias}"),
        (F.col(f"databricks.{agg_alias}") - F.col(f"sql.{agg_alias}")).alias("Difference")
    )

    # Mostrar os resultados
    comparison_df.display()

# ----- CONSULTAS -----

# 1. Produtos Mais Vendidos (Quantidade Total)
compare_aggregations(
    databricks_df=df_databricks,
    sql_df=df_sql,
    group_by_cols=["ProductName", "ProductCategoryName", "ProductSubcategoryName"],
    agg_col="OrderQty",
    agg_func=F.sum,
    agg_alias="TotalQuantitySold"
)

# 2. Produtos com Maior Receita Total
compare_aggregations(
    databricks_df=df_databricks,
    sql_df=df_sql,
    group_by_cols=["ProductName", "ProductCategoryName", "ProductSubcategoryName"],
    agg_col="LineTotal",
    agg_func=F.sum,
    agg_alias="TotalRevenue"
)

# 3. Receita por Categoria
compare_aggregations(
    databricks_df=df_databricks,
    sql_df=df_sql,
    group_by_cols=["ProductCategoryName"],
    agg_col="LineTotal",
    agg_func=F.sum,
    agg_alias="TotalRevenue"
)

# 4. Subcategorias com Maior Venda em Volume
compare_aggregations(
    databricks_df=df_databricks,
    sql_df=df_sql,
    group_by_cols=["ProductSubcategoryName", "ProductCategoryName"],
    agg_col="OrderQty",
    agg_func=F.sum,
    agg_alias="TotalQuantity"
)

# 5. Clientes que Mais Compraram (Receita)
compare_aggregations(
    databricks_df=df_databricks,
    sql_df=df_sql,
    group_by_cols=["CustomerName"],
    agg_col="LineTotal",
    agg_func=F.sum,
    agg_alias="TotalCustomerRevenue"
)

# 6. Produtos com Maior Desconto Aplicado
compare_aggregations(
    databricks_df=df_databricks.filter(F.col("UnitPriceDiscount") > 0),
    sql_df=df_sql.filter(F.col("UnitPriceDiscount") > 0),
    group_by_cols=["ProductName", "ProductCategoryName", "ProductSubcategoryName"],
    agg_col="UnitPriceDiscount",
    agg_func=F.sum,
    agg_alias="TotalDiscountGiven"
)

# 7. Tendência Mensal de Vendas (Receita)
compare_aggregations(
    databricks_df=df_databricks,
    sql_df=df_sql,
    group_by_cols=[F.year("OrderDate").alias("Year"), F.month("OrderDate").alias("Month")],
    agg_col="LineTotal",
    agg_func=F.sum,
    agg_alias="MonthlyRevenue"
)
